## 환경 설정

In [1]:
# !pip install mxnet
# !pip install torch==1.10.1
# !pip install transformers==4.8.2
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece==0.1.91
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-71s2rpdb
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-71s2rpdb
     |████████████████████████████████| 129 kB 5.1 MB/s 
     |████████████████████████████████| 344 kB 41.4 MB/s 
     |████████████████████████████████| 54.7 MB 1.1 MB/s 
     |████████████████████████████████| 4.5 MB 48.2 MB/s 
     |████████████████████████████████| 1.2 MB 48.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:43tcmalloc: large alloc 1147494400 bytes == 0x3a430000 @  0x7f5d17070615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 

In [31]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import gluonnlp as nlp
from tqdm.notebook import tqdm

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

device = torch.device("cuda:0")

In [3]:
device

device(type='cuda', index=0)

# 데이터 불러오기

In [4]:
data = pd.read_csv("/content/naverReview.csv", encoding='utf-8')
data

,star,review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
162381,5,삼실에서 쓰는데 너무 좋구여! 한가지 팁은 집에 갈때 물통을 비우고 말려주고 다시 ...
162382,5,무료영작 첨삭 받아서 과제 제출 했습니다 글자수를좀더 늘려주세요
162383,2,전에 샀던 제품과 질이 너무달라요 컨버스 질도 그렇고 붓도 그렇고 타사이트에서 구입...
162384,4,여러개샀는데 대마음에들어용 담에 또 신상들어오면 사야겠어요


# 0과 1로 레이블 변경

In [5]:
def changeTo01(x):
  if x<3:
    return 0
  else:
    return 1

data['star'] = data['star'].apply(changeTo01)
data

,star,review
0,1,배공빠르고 굿
1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,1,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,0,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,1,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
162381,1,삼실에서 쓰는데 너무 좋구여! 한가지 팁은 집에 갈때 물통을 비우고 말려주고 다시 ...
162382,1,무료영작 첨삭 받아서 과제 제출 했습니다 글자수를좀더 늘려주세요
162383,0,전에 샀던 제품과 질이 너무달라요 컨버스 질도 그렇고 붓도 그렇고 타사이트에서 구입...
162384,1,여러개샀는데 대마음에들어용 담에 또 신상들어오면 사야겠어요


In [6]:
data['star'].value_counts()

0    81362
1    81024
Name: star, dtype: int64

# Input for BERTDataset

In [7]:
data_list = []

for review, label in zip(data['review'], data['star']):
  data = []
  data.append(review)
  data.append(label)
  data_list.append(data)

len(data_list)

162386

In [8]:
data_list[:5]

[['배공빠르고 굿', 1],
 ['택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고', 0],
 ['아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.', 1],
 ['선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..',
  0],
 ['민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ', 1]]

# Train, Test Split

In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_list, test_size = 0.2, shuffle=True, random_state = 0)

In [10]:
print(len(train))
train[:5]

129908


[['호불호가 갈릴거 같음', 0],
 ['재구매 빠른 배송에 매우 만족합니다. 적극 추천합니다. 착한가격에 제품도 좋구..항상 재구매하고있습니다. 수고하세여', 1],
 ['시계 원래 줄 너무 두꺼워서 별로였는데 이건아주 좋음', 1],
 ['리뷰보면 남들차는 잘만지웠졌던데 제 차만 하나도 안 지워지는건가요? 바를때와 바르고 닦은 후랑 차이가 하나도 없어요 벗겨지지도 않았는데 ㅜㅜ 개봉했으니 반품도 못하고 ㅜㅜ 나만 그런가? 에효',
  0],
 ['옷은 너무 이쁜데 배송기간이랑 우체국 택배 완전 별로네요', 0]]

In [11]:
print(len(test))
test[:5]

32478


[['튼튼하지 안꾸 흔들려요', 0],
 ['다른게임보다 웃을수있는건 작고 가격대비 별로예요...', 0],
 ['원래 아이가 좋아하는 간식인데 할인받아서 저렴하게 샀어요!', 1],
 ['정말 만족스럽네요 고급 스럽고 좋습니다~', 1],
 ['새 나이프 인데도 잘 안들어요..좀 실망요.', 0]]

## KoBERT tokenizer, model

[CLS] : classification token (문장의 시작)

[SEP] : seperate (문장 구분)

[MASK] : MASK

[PAD] : PADDING

In [12]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [13]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [14]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
tok.vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [16]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

# Dataset

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# Parameter

In [18]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [19]:
# for i in train:
#   print(i[0])
#   print(i[1])
#   break

In [20]:
train_dataset = BERTDataset(train, 0, 1, tok, max_len, True, False)
test_dataset = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [21]:
train_dataset.labels[:3]

[0, 1, 1]

In [22]:
test_dataset.sentences[:3]

[(array([   2,  517, 7668, 7668, 7819, 3135, 5628, 5199, 6060, 6999,    3,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
  array(11, dtype=int32),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=int32)),
 (array([   2, 1567, 5401, 6371, 3528, 7088, 6629, 7142, 5384, 3930, 5439,
          728, 5814,  517, 6357, 6957, 6999,  517,   54,  517,   54,  517,
           54,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,   

# DataLoader

In [23]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=5)

# KoBERT 모델

In [29]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        out = self.dropout(pooler)
    else:
        out = pooler
    return self.classifier(out)

In [30]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer and schedule

In [36]:
no_decay = ['bias', 'LayerNorm.weight']

# 최적화해야 할 parameter를 optimizer에게 알려야 합
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [45]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate) # optimizer
loss_fn = nn.CrossEntropyLoss() # loss function 

In [39]:
t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [42]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# Calculate Accuracy

In [43]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# Train

In [46]:
for e in range(num_epochs):
    
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_loader), total=len(test_loader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/2030 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7444978952407837 train acc 0.546875
epoch 1 batch id 201 loss 0.38254767656326294 train acc 0.6871890547263682
epoch 1 batch id 401 loss 0.17826028168201447 train acc 0.791341957605985
epoch 1 batch id 601 loss 0.26687440276145935 train acc 0.8321287437603994
epoch 1 batch id 801 loss 0.2501623332500458 train acc 0.8520989388264669
epoch 1 batch id 1001 loss 0.1882733553647995 train acc 0.8655563186813187
epoch 1 batch id 1201 loss 0.27335137128829956 train acc 0.8742584304746045
epoch 1 batch id 1401 loss 0.1684371829032898 train acc 0.8809109564596717
epoch 1 batch id 1601 loss 0.26554858684539795 train acc 0.885657401623985
epoch 1 batch id 1801 loss 0.1662771999835968 train acc 0.8901044558578567
epoch 1 batch id 2001 loss 0.30484944581985474 train acc 0.8937952898550725
epoch 1 train acc 0.894095182834407


  0%|          | 0/508 [00:00<?, ?it/s]

epoch 1 test acc 0.9300524934383202


  0%|          | 0/2030 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9317441765091863


  0%|          | 0/2030 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.25370681285858154 train acc 0.921875
epoch 5 batch id 201 loss 0.32785099744796753 train acc 0.9745802238805971
epoch 5 batch id 401 loss 0.09999707341194153 train acc 0.9734258104738155
epoch 5 batch id 601 loss 0.09405636787414551 train acc 0.9754055740432612
epoch 5 batch id 801 loss 0.06409438699483871 train acc 0.9759675405742821
epoch 5 batch id 1001 loss 0.05548015609383583 train acc 0.9771010239760239
epoch 5 batch id 1201 loss 0.14370259642601013 train acc 0.9773496044962531
epoch 5 batch id 1401 loss 0.07559092342853546 train acc 0.977471448965025
epoch 5 batch id 1601 loss 0.1111510843038559 train acc 0.9778263585259213
epoch 5 batch id 1801 loss 0.06502994894981384 train acc 0.9781458217656858
epoch 5 batch id 2001 loss 0.24605774879455566 train acc 0.9780890804597702
epoch 5 train acc 0.9780290829859796


  0%|          | 0/508 [00:00<?, ?it/s]

epoch 5 test acc 0.9315247703412074


# Input Test

In [53]:
def predict(predict_sentence):

    data = [predict_sentence, 0]
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_loader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_loader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정적")
            else:
                test_eval.append("긍정적")

        print(">> 해당 리뷰는 " + test_eval[0] + " 리뷰 입니다.")

In [57]:
while True:
    sentence = input("긍부정을 판단할 리뷰를 입력해주세요 : ")
    if sentence == "0":
      print(">> 긍부정 판단을 종료합니다!")
      break
    predict(sentence)
    print("\n")

긍부정을 판단할 리뷰를 입력해주세요 : 박스가 다 터져왔어요
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 이렇게 장사하지 마세요 😭
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 물건 상태가 너무 좋네요~~
>> 해당 리뷰는 긍정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 0
>> 긍부정 판단을 종료합니다!
